In [2]:
# The programming language we are using is python
# We require the use of some functions not already part of this language
# Instead of writing them from scratch, we can import them as part of external libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
# This is the file that contains the nuclear genome data
nuclear = pd.read_csv("C:/Users/natha/Desktop/MSCi/nucleardata.csv")

In [4]:
# We will preview the first 5 entries to see what they look like
nuclear.head()

,Transcript name,5UTR,3UTR,Mettler C0,Tertile
0,ADC1,ATAAAGGTCGGGCCGGTTGTTTACAATGAAGGGCACAGCCGCGCCA...,AGTGGACTGTCGGCATAAGAATGCTGTGCTAGGAATAGGTACCGTG...,9.527054,2
1,LHCSR1,AGAGTCATTCCCCAACCCACTTCTCAGCTTGCACCTCCTTGTGCAC...,GTGCCTCTTCCTCGATCGTGATCTGCTTCACGGTTTGGTCGCGGGC...,12.158357,3
2,GRX2,CACATTGGCACTGGAGTTTATTTGAGGCGATACATGCCTTCTCATG...,GGCGAGGGCGGCGGGGTGGGCAGCAGGTTGTGAGGCGCGTGCAGGG...,14.064211,3
3,TIM16,GCACCAAACTTTCCCTGGGCCCCCCTCTGGGCGCACTCGAAATGCT...,CGAGGCGTGGCGTGAGGTGACAAGCAGGGGCGATGGATGTTGGGGA...,8.464260,2
4,Cre01.g056200,ACTTGTCAAAACCTTGCAAGGCTTGCGGCAGGCACTGTCCAGTACC...,TGGGTCAGTGGGGCGTGTGTGTGGCCGGCTGCTGGACCCGTGAGCA...,14.582071,3


In [5]:
# Confirming we have no empty fields
nuclear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Transcript name  384 non-null    object 
 1   5UTR             384 non-null    object 
 2   3UTR             384 non-null    object 
 3   Mettler C0       384 non-null    float64
 4   Tertile          384 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 15.1+ KB


In [6]:
# These are the columns of our data
nuclear.keys()

Index(['Transcript name', '5UTR', '3UTR', 'Mettler C0', 'Tertile'], dtype='object')

In [7]:
# As with the other file, this function will split our DNA sequences into smaller lengths called k-mers for processing
def kmerconvert (sequence, k_length):
    k_conversion = []
    num_kmers = len(sequence) - k_length + 1
    
    for element in range(num_kmers):
        kmer = sequence[element:element + k_length]
        k_conversion.append(kmer)
        
    return k_conversion

In [8]:
# Applying this function to our 3' UTRs
nuclear['3UTRK'] = nuclear.apply(lambda x: kmerconvert(x['3UTR'], k_length=3), axis=1)

In [9]:
# Applying this function to our 5' UTRs
nuclear['5UTRK'] = nuclear.apply(lambda x: kmerconvert(x['5UTR'], k_length=3), axis=1)

In [10]:
# We can see the effect of this below
nuclear.head()

,Transcript name,5UTR,3UTR,Mettler C0,Tertile,3UTRK,5UTRK
0,ADC1,ATAAAGGTCGGGCCGGTTGTTTACAATGAAGGGCACAGCCGCGCCA...,AGTGGACTGTCGGCATAAGAATGCTGTGCTAGGAATAGGTACCGTG...,9.527054,2,"[AGT, GTG, TGG, GGA, GAC, ACT, CTG, TGT, GTC, ...","[ATA, TAA, AAA, AAG, AGG, GGT, GTC, TCG, CGG, ..."
1,LHCSR1,AGAGTCATTCCCCAACCCACTTCTCAGCTTGCACCTCCTTGTGCAC...,GTGCCTCTTCCTCGATCGTGATCTGCTTCACGGTTTGGTCGCGGGC...,12.158357,3,"[GTG, TGC, GCC, CCT, CTC, TCT, CTT, TTC, TCC, ...","[AGA, GAG, AGT, GTC, TCA, CAT, ATT, TTC, TCC, ..."
2,GRX2,CACATTGGCACTGGAGTTTATTTGAGGCGATACATGCCTTCTCATG...,GGCGAGGGCGGCGGGGTGGGCAGCAGGTTGTGAGGCGCGTGCAGGG...,14.064211,3,"[GGC, GCG, CGA, GAG, AGG, GGG, GGC, GCG, CGG, ...","[CAC, ACA, CAT, ATT, TTG, TGG, GGC, GCA, CAC, ..."
3,TIM16,GCACCAAACTTTCCCTGGGCCCCCCTCTGGGCGCACTCGAAATGCT...,CGAGGCGTGGCGTGAGGTGACAAGCAGGGGCGATGGATGTTGGGGA...,8.464260,2,"[CGA, GAG, AGG, GGC, GCG, CGT, GTG, TGG, GGC, ...","[GCA, CAC, ACC, CCA, CAA, AAA, AAC, ACT, CTT, ..."
4,Cre01.g056200,ACTTGTCAAAACCTTGCAAGGCTTGCGGCAGGCACTGTCCAGTACC...,TGGGTCAGTGGGGCGTGTGTGTGGCCGGCTGCTGGACCCGTGAGCA...,14.582071,3,"[TGG, GGG, GGT, GTC, TCA, CAG, AGT, GTG, TGG, ...","[ACT, CTT, TTG, TGT, GTC, TCA, CAA, AAA, AAA, ..."


In [11]:
# This will allow our system to input both 3' and 5' UTRs together
nuclear["Combined"] = nuclear["3UTRK"] + nuclear["5UTRK"]

In [12]:
# This function will combine the k-mers (subsequences of each UTR) into "sentences" such that they may be processed in groups
kmer_sentences = list(nuclear['Combined'])
for e in range(len(kmer_sentences)):
    kmer_sentences[e] = ' '.join(kmer_sentences[e])

In [13]:
# This will count the frequency of each subsequence (k-mer) in each UTR region
cv = CountVectorizer(ngram_range=(8,8))
X = cv.fit_transform(kmer_sentences)
print(X.shape)

(384, 283045)


In [14]:
# Setting the prediction target as tertile of gene expression
y = nuclear[['Tertile']]

In [15]:
# We will now train 2 classifier algorithms on all of this data
# There will be no test/train split because we want to train it with as much data as possible
# This means it will make the most accurate predictions on the chloroplast gene dataset

In [16]:
# Training a multinomial naive Bayes classifier
#System 3 model 1, multinomial naive Bayes
from sklearn.naive_bayes import MultinomialNB
nuclearclassifier = MultinomialNB(alpha = 0.0001)
nuclearclassifier.fit(X, np.ravel(y))

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [17]:
# Training a random forest classifier
# System 3 model 2, random forest 
from sklearn.ensemble import RandomForestClassifier
nuclearclassifier2 = RandomForestClassifier()
nuclearclassifier2.fit(X, np.ravel(y))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
# We will now specify the location of the chloroplast data
chloroplast = pd.read_csv("C:/Users/natha/Desktop/MSCi/algae.csv")

In [19]:
# We will again preview the data, and check to ensure there are no empty fields
chloroplast.head()
chloroplast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gene            67 non-null     object 
 1   Average         67 non-null     float64
 2   Normalised avg  67 non-null     float64
 3   3UTR            67 non-null     object 
 4   5UTR            67 non-null     object 
 5   Quintile        67 non-null     int64  
 6   Tertile         67 non-null     int64  
 7   3UTR MFE        67 non-null     float64
 8   3UTRL           67 non-null     int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 4.8+ KB


In [20]:
# As above, splitting our 3' UTRs into smaller subsequences
chloroplast['3UTRK'] = chloroplast.apply(lambda x: kmerconvert(x['3UTR'], k_length=3), axis=1)

In [21]:
# Now the same for our 5' UTRs
chloroplast['5UTRK'] = chloroplast.apply(lambda x: kmerconvert(x['5UTR'], k_length=3), axis=1)

In [22]:
# Joining the 3' and 5' UTR sequences so that they may be input together into our machine learning systems
chloroplast["Combined"] = chloroplast["3UTRK"] + chloroplast["5UTRK"]

In [23]:
# This changes the way that the k-mers are stored, as above
kmer_sentences2 = list(chloroplast['Combined'])
for e in range(len(kmer_sentences2)):
    kmer_sentences2[e] = ' '.join(kmer_sentences2[e])

In [24]:
# We will compute the frequency of each k-mer in the UTRs
X2 = cv.transform(kmer_sentences2)
print(X2.shape)

(67, 283045)


In [25]:
# Previews the first 5 genes in the chloroplast plastome dataset
chloroplast.head()

,Gene,Average,Normalised avg,3UTR,5UTR,Quintile,Tertile,3UTR MFE,3UTRL,3UTRK,5UTRK,Combined
0,atpA,11.88,0.832,tttttaattaagtaggaactcggtatatgctcttttggggtcttat...,attttaatgcttatgctatcttttttatttagtccataaaaccttt...,5,3,-31.3,185,"[ttt, ttt, ttt, tta, taa, aat, att, tta, taa, ...","[att, ttt, ttt, tta, taa, aat, atg, tgc, gct, ...","[ttt, ttt, ttt, tta, taa, aat, att, tta, taa, ..."
1,atpB,11.62,0.814,ttggctctttaagaagaaaacaacttaatggtgtccaaatatttat...,tcatattttaacttattttacttaaattcttacgtataaaccccga...,5,3,-4.9,70,"[ttg, tgg, ggc, gct, ctc, tct, ctt, ttt, tta, ...","[tca, cat, ata, tat, att, ttt, ttt, tta, taa, ...","[ttg, tgg, ggc, gct, ctc, tct, ctt, ttt, tta, ..."
2,atpE,11.36,0.796,ttagatctatgtatttacccaaagagtatactgttcaactctatca...,aataaaaaaaaataaaacttctaaaagcgataaagctagaacattc...,4,3,-30.1,132,"[tta, tag, aga, gat, atc, tct, cta, tat, atg, ...","[aat, ata, taa, aaa, aaa, aaa, aaa, aaa, aaa, ...","[tta, tag, aga, gat, atc, tct, cta, tat, atg, ..."
3,atpF,9.68,0.678,tttttaataaagcttactaacgaattacactaactttactgtattt...,aaataaatttgacttagcttaaatttcagtatatttatatacacaa...,3,2,-230.6,628,"[ttt, ttt, ttt, tta, taa, aat, ata, taa, aaa, ...","[aaa, aat, ata, taa, aaa, aat, att, ttt, ttg, ...","[ttt, ttt, ttt, tta, taa, aat, ata, taa, aaa, ..."
4,atpH,13.85,0.971,ttttaaatttacatgttgtaaaggatatctatagaaatgggagtta...,tcggataaaacatattatgaccgtataatgtttttccaccattgaa...,5,3,-206.4,986,"[ttt, ttt, tta, taa, aaa, aat, att, ttt, tta, ...","[tcg, cgg, gga, gat, ata, taa, aaa, aaa, aac, ...","[ttt, ttt, tta, taa, aaa, aat, att, ttt, tta, ..."


In [32]:
# Sets the prediction target as tertle of expression for the chloroplast genes, and makes the predictions
y2 = chloroplast[['Tertile']]
y2_pred = np.empty(67,) 

In [33]:
# This is for the multinomial naive Bayes classifier
y2_pred = nuclearclassifier.predict(X2)
print("System 3 model 1 has an accuracy of " + str(round((accuracy_score(y2_pred, y2)),2)*100) +"%")
print("System 3 model 1 has a precision of " + str(round((precision_score(y2_pred, y2, average='weighted')),2)*100) +"%")
print("System 3 model 1 has a recall score of " + str(round((recall_score(y2_pred, y2, average='weighted')),2)*100) +"%")

System 3 model 1 has an accuracy of 46.0%
System 3 model 1 has a precision of 63.0%
System 3 model 1 has a recall score of 46.0%


In [45]:
# This is for the random forest classifier
y3_pred = nuclearclassifier2.predict(X2)
print("System 3 model 2 has an accuracy of " + str(round((accuracy_score(y3_pred, y2)),2)*100) +"%")
print("System 3 model 2 has a precision of " + str(round((precision_score(y3_pred, y2, average='weighted')),2)*100) +"%")
print("System 3 model 2 has a recall score of " + str(round((recall_score(y3_pred, y2, average='weighted')),2)*100) +"%")
print("Having low accuracy and recall scores but a very high precision score seems erroneous")

System 3 model 2 has an accuracy of 33.0%
System 3 model 2 has a precision of 100.0%
System 3 model 2 has a recall score of 33.0%
Having low accuracy and recall scores but a very high precision score seems erroneous


In [48]:
print("Let's have a look at what system 3 model 2 predicted")
print(y3_pred)
print("The random forest classifier has predicted expression tertile 2 for all genes: this is clearly innacurate!")
print("Clearly the multinomial naive Bayes model (system 3 model 1) is superior")

Let's have a look at what system 3 model 2 predicted
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
The random forest classifier has predicted expression tertile 2 for all genes: this is clearly innacurate!
Clearly the multinomial naive Bayes model (system 3 model 1) is superior
